In [56]:

%pip install pyserini==0.12.0
%pip install glove-python-binary
%pip install glove

import numpy as np
import json
import re
from urllib.request import urlopen
import nltk
from pyserini import search
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

np.random.seed(1)


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement glove-python-binary (from versions: none)
ERROR: No matching distribution found for glove-python-binary

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached glove-1.0.2.tar.gz (44 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-12-arm64-cpython-310
      creating build/lib.macosx-12-arm64-cpython-310/glove
      copying glove/__ini

[nltk_data] Downloading package wordnet to /Users/hstack/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/hstack/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hstack/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
docids = set()
# queryId -> docId -> relevanceScore
qrels = dict()
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  new_docid = docid.decode('UTF-8')
  docids.add(new_docid)

  if int(qid) in qrels:
    qrels[int(qid)][new_docid] = int(score)
  else:
    qrels[int(qid)] = dict()
    qrels[int(qid)][new_docid] = int(score)


docids = list(docids)
np.random.shuffle(docids)
print(len(docids), docids[0])
print(len(qrels))

174787 FBIS4-32573
249


In [58]:
data = []
searcher = search.SimpleSearcher.from_prebuilt_index('robust04')
for docid in docids[:50]:
  try:
    text = searcher.doc(docid).raw()
    data.append((docid, text))
  except:
    continue

cleaned_data = []
for docid, text in data:
  new_text = text.replace('<P>', ' ')
  new_text = new_text.replace('</P>', ' ')
  new_text = new_text.replace('<DATE>', ' ')
  new_text = new_text.replace('</DATE>', ' ')
  new_text = new_text.replace('<HEADLINE>', ' ')
  new_text = new_text.replace('</HEADLINE>', ' ')
  new_text = new_text.replace('<TEXT>', ' ')
  new_text = new_text.replace('</TEXT>', ' ')
  new_text = new_text.replace('\n', ' ')
  new_text = new_text.replace("\'", '')
  new_text = new_text.replace('--', ' ')
  cleaned_data.append((docid, new_text))
  
cleaned_data[0]

Attempting to initialize pre-built index robust04.
/Users/hstack/.cache/pyserini/indexes/index-robust04-20191213.15f3d001489c97849a010b0a4734d018 already exists, skipping download.
Initializing robust04...


('FBIS4-32573',
 '  BFN  <F P=106> [Report by editor Tomas Sulak on the second regular congress </F> of the Christian Social Union in Moravia on 25 June recorded] [Text] [Sulak] The Christian Social Union [KSU], formed by the opposition to the leadership of the Christian Democratic Union-Czechoslovakia Peoples Party [KDU-CSL], headed by Josef Lux, held a congress today. The participants are in favor of following the spirit of the prewar Czechoslovak Peoples Party headed by Jan Sramek. Milos Hanak, former member of the Peoples Party and the current chairman of the Vyskov, Moravia District privatization commission, was elected the new chairman of the party. Four new deputy chairmen and a new central committee of the party were also elected. The congress stripped former party Chairman Frantisek Michalek of all party posts. According to its new chairman, Hanak, the KSU is currently operating within the framework of the so-called Realistic Bloc, considers itself to be a party of the center,

In [59]:
stemmer = nltk.stem.PorterStemmer()
stop_words = set(nltk.corpus.stopwords.words('english'))

def tokenize(doc_id_and_text):
    text = doc_id_and_text[1]
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    tokens = pattern.findall(text)
    stemmed_tokens = []
    for token in tokens:
      stemmed_token = stemmer.stem(token.replace("'", ''))
      if stemmed_token not in stop_words:
        stemmed_tokens.append(stemmed_token)
    return (doc_id_and_text[0],stemmed_tokens)

tokens_by_doc = [tokenize(doc) for doc in cleaned_data[:25]]
print(tokens_by_doc[0])

doc_id_to_tokens = dict()
for tokens in tokens_by_doc:
  doc_id_to_tokens[tokens[0]] = tokens[1]

('FBIS4-32573', ['bfn', 'f', 'p', 'report', 'editor', 'toma', 'sulak', 'second', 'regular', 'congress', 'f', 'christian', 'social', 'union', 'moravia', 'june', 'record', 'text', 'sulak', 'christian', 'social', 'union', 'ksu', 'form', 'opposit', 'leadership', 'christian', 'democrat', 'union', 'czechoslovakia', 'peopl', 'parti', 'kdu', 'csl', 'head', 'josef', 'lux', 'held', 'congress', 'today', 'particip', 'favor', 'follow', 'spirit', 'prewar', 'czechoslovak', 'peopl', 'parti', 'head', 'jan', 'sramek', 'milo', 'hanak', 'former', 'member', 'peopl', 'parti', 'current', 'chairman', 'vyskov', 'moravia', 'district', 'privat', 'commiss', 'wa', 'elect', 'new', 'chairman', 'parti', 'four', 'new', 'deputi', 'chairmen', 'new', 'central', 'committe', 'parti', 'also', 'elect', 'congress', 'strip', 'former', 'parti', 'chairman', 'frantisek', 'michalek', 'parti', 'post', 'accord', 'new', 'chairman', 'hanak', 'ksu', 'current', 'oper', 'within', 'framework', 'call', 'realist', 'bloc', 'consid', 'parti',

In [120]:
# Use the topics from the homework to search by document
from pyserini.search import get_topics
topics = get_topics('robust04')
print(len(topics))

idx = 0
for topic in topics:
    if idx < 5:
        print(topic)
        print(topics[topic]['title'])
    else:
        break

250
350
Health and Computer Terminals
351
Falkland petroleum exploration
352
British Chunnel impact
353
Antarctica exploration
354
journalist risks
355
ocean remote sensing
356
postmenopausal estrogen Britain
357
territorial waters dispute
358
blood-alcohol fatalities
359
mutual fund predictors
360
drug legalization benefits
361
clothing sweatshops
362
human smuggling
363
transportation tunnel disasters
364
rabies
365
El Nino
366
commercial cyanide uses
367
piracy
368
in vitro fertilization
369
anorexia nervosa bulimia
370
food/drug laws
371
health insurance holistic
372
Native American casino
373
encryption equipment export
374
Nobel prize winners
375
hydrogen energy
376
World Court
377
cigar smoking
378
euro opposition
379
mainstreaming
380
obesity medical treatment
381
alternative medicine
382
hydrogen fuel automobiles
383
mental illness drugs
384
space station moon
385
hybrid fuel cars
386
teaching disabled children
387
radioactive waste
388
organic soil enhancement
389
illegal tec

In [61]:
import json

with open('embeddings.json') as f:
    word_embeddings = json.load(f)
print(word_embeddings['octob'])

[0.8538881190934323, 0.3578883330164455, -0.1872723552896967, -0.3523864200200205, -0.46351744368155007, 0.6468629776968987, 0.6614826316673852, -0.3668020283483838, 0.9602728314242588, 0.3856306968482955]


In [62]:
# We need to get the embeddings for every word in each document document (you do np.mean() on the embeddings list to do this)
import numpy as np
import nltk

VECTOR_SIZE = 10
stemmer = nltk.stem.PorterStemmer()

def get_document_embedding(doc_tokens):
    embeddings = []
    if len(doc_tokens) < 1:
        return np.zeros(VECTOR_SIZE)
    else:
        for token in doc_tokens:
            stemmed_token = stemmer.stem(token)
            if stemmed_token in word_embeddings:
                embeddings.append(word_embeddings[stemmed_token])
            else:
                embeddings.append(np.random.normal(0, 1, VECTOR_SIZE))
        # average the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

document_embeddings = dict()
for doc in tokens_by_doc:
    document_embeddings[doc[0]] = get_document_embedding(doc[1])

In [118]:
# When given a query, get its embedding and then do cosine similarity with each of the embedding of the documents
import heapq

def cosim(v1, v2):
  return np.dot(v1, v2)/(np.linalg.norm(v1) * np.linalg.norm(v2))

def get_our_top_hundred_documents(query):
    # Maybe do something different for query tokens 
    query_tokens = query.split(' ')
    query_embedding = get_document_embedding(query_tokens)
    
    similarities = []

    idx = 0
    for document in document_embeddings:
      # Will prob have to change this
      document_name = document
      document_embedding = document_embeddings[document]
      similarity = cosim(document_embedding, query_embedding)
      if idx < 100:
        heapq.heappush(similarities, (similarity, document_name))
      else:
        heapq.heapreplace(similarities, (similarity, document_name))
      idx += 1
    return heapq.nlargest(100, similarities)

def get_top_hundred_fast_text_documents(query):
  hits = searcher.search(query, 100)
  return [(hit.score, hit.docid) for hit in hits]


def get_top_hundred_ctrl_plus_f_documents(query):
  relevant_docs = set()

  query_tokens = []
  for token in query.split(' '):
    query_tokens.append(stemmer.stem(token))
  for tup in doc_id_to_tokens:
    doc_id = tup[0]
    tokens = tup[1]
    # last_token_index = len(tokens) - 1
    # current_token_index = 0
    occurences = 0

    for token in tokens:
      if stemmer.stem(token) in query_tokens:
        occurences += 1
      # if token == query_tokens[current_token_index]:
      #   if current_token_index == last_token_index:
      #     occurences += 1
      #     current_token_index = 0
          
      #   else:
      #     current_token_index += 1
    if occurences > 0:
      relevant_docs.add((doc_id, occurences))
    doc_list = list(relevant_docs)
    doc_list.sort(key=lambda x: x[1], reverse=True)
    return doc_list
        



In [114]:
# topic is a qid -> dict['title] is the query
first_query_ranking = get_our_top_hundred_documents(topics[301]['title'])
for i in range(10):
    print(first_query_ranking[i])
print('------')


(0.9631266454888019, 'FBIS4-18004')
(0.9228843446963603, 'LA052090-0019')
(0.9202195824039938, 'FBIS3-1476')
(0.9133623374711642, 'FBIS4-50071')
(0.9017937317931818, 'FBIS4-43047')
(0.9000857829795657, 'FT934-16490')
(0.8979535928831708, 'FT943-1007')
(0.8895116017459416, 'FBIS3-59833')
(0.8843022866862087, 'FT934-2731')
(0.8815653978588323, 'FT942-13766')
------


In [115]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

In [116]:
def get_relevant_docs(query_id):
    relevant_docs = set()
    if query_id in qrels:
        for document_id in qrels[query_id]:
            if document_id in qrels[query_id]:
                if qrels[query_id][document_id] == 1:
                    relevant_docs.add(document_id)
    return relevant_docs

# Calculate the MAP@100 for our model, CTRL + F, fasttext, and GloVe
def map_hundred(given_topics, model_type):
    average_precision_values = []
    for topic in given_topics:
        query = topics[topic]['title']
        actual_relevant_docs = get_relevant_docs(topic)
        if model_type == "ours":
            our_relevant_docs = get_our_top_hundred_documents(query)
        elif model_type == "fasttext":
            our_relevant_docs = get_top_hundred_fast_text_documents(query)
        elif model_type == 'ctrl-f':
            our_relevant_docs = get_top_hundred_ctrl_plus_f_documents(query)
        
        summed_precision = 0
        relevant_count = 0
        total_count = 0
        for i in range(len(our_relevant_docs)):
            should_add = False
            if our_relevant_docs[i][1] in actual_relevant_docs:
                relevant_count += 1
                should_add = True
            total_count += 1
            if should_add:
                summed_precision += (relevant_count / total_count)
        if relevant_count > 0:
            average_precision_values.append(summed_precision / relevant_count)
    average_sum = sum(average_precision_values)
    if len(average_precision_values) > 0:
        return average_sum / len(average_precision_values)
    return 0


In [119]:
our_map_at_hundred = map_hundred(test_topics, 'ours')
print(our_map_at_hundred)
fasttext_map_at_hundred = map_hundred(test_topics, 'fasttext')
print(fasttext_map_at_hundred)
ctrl_f_map_at_hundred = map_hundred(test_topics, 'ctrl-f')
print(ctrl_f_map_at_hundred)

0
0.4133287942842544
0


In [121]:
# relevant docs for 350, 351, 352
relevant_docs = []
for doc_id in qrels[350]:
    if qrels[350][doc_id] == 1:
        relevant_docs.append(doc_id)
print(relevant_docs)
print("---------")
relevant_docs = []
for doc_id in qrels[351]:
    if qrels[351][doc_id] == 1:
        relevant_docs.append(doc_id)
print(relevant_docs)
print("---------")
relevant_docs = []
for doc_id in qrels[352]:
    if qrels[352][doc_id] == 1:
        relevant_docs.append(doc_id)
print(relevant_docs)
print("---------")

['FBIS3-40544', 'FBIS3-43068', 'FBIS4-47430', 'FR940405-1-00038', 'FT921-10581', 'FT921-11676', 'FT921-11926', 'FT921-12737', 'FT921-12910', 'FT921-14780', 'FT923-11262', 'FT923-1719', 'FT923-4291', 'FT924-15322', 'FT924-5774', 'FT924-5983', 'FT924-6634', 'FT932-12233', 'FT932-14135', 'FT932-451', 'FT932-531', 'FT932-8474', 'FT933-10020', 'FT933-12004', 'FT933-14778', 'FT933-1948', 'FT934-10688', 'FT934-11654', 'FT934-11789', 'FT934-11852', 'FT934-16045', 'FT941-16077', 'FT941-429', 'FT941-7300', 'FT941-7564', 'FT942-9881', 'FT943-4711', 'FT943-5077', 'FT943-8342', 'FT944-13668', 'FT944-9163', 'LA011389-0085', 'LA011989-0198', 'LA012590-0139', 'LA030389-0012', 'LA031289-0009', 'LA031990-0076', 'LA032190-0134', 'LA032789-0066', 'LA052290-0188', 'LA053090-0082', 'LA060690-0112', 'LA062289-0031', 'LA062390-0104', 'LA070290-0099', 'LA082990-0081', 'LA083189-0200', 'LA090589-0110', 'LA091090-0108', 'LA091589-0052', 'LA100690-0107', 'LA112889-0127', 'LA120689-0152', 'LA120789-0021', 'LA12108